In [1]:
# Import necessary libraries, install them if needed
# For compiling, you will need Jupyter pip package installed.

from sample_data import price_lists
from typing import Dict, Tuple, Optional
from functools import lru_cache
import json

In [2]:
def clean_price_lists(price_lists: Dict[str, Dict[str, float]]) -> Dict[str, Dict[str, float]]:
    """
    Clean the price lists by removing any corrupt data entries.
    
    Args:
    price_lists (dict): A dictionary containing the price lists of operators.
    
    Returns:
    dict: A cleaned dictionary with valid price entries.
    """
    cleaned_lists = {}
    for operator, prices in price_lists.items():
        cleaned_prices = {}
        for prefix, price in prices.items():
            if isinstance(price, (int, float)) and price >= 0 and prefix.replace('.', '', 1).isdigit():
                cleaned_prices[prefix] = price
            else:
                print(f"Removed invalid entry from {operator}: prefix={prefix}, price={price}")
        cleaned_lists[operator] = cleaned_prices
    return cleaned_lists

In [3]:
def preprocess_price_lists(price_lists: Dict[str, Dict[str, float]]) -> Dict[str, Dict[str, float]]:
    """
    Preprocess the price lists to allow for faster lookups.

    Args:
    price_lists (dict): A dictionary containing the price lists of operators.

    Returns:
    dict: A dictionary where keys are operators and values are dictionaries of prefixes and prices.
    """
    preprocessed = {}
    for operator, prices in price_lists.items():
        preprocessed[operator] = {prefix: price for prefix, price in prices.items() if isinstance(price, (int, float)) and price >= 0}
    return preprocessed

In [4]:
def dict_to_json_str(d: Dict) -> str:
    """
    Convert a dictionary to a JSON string.
    
    Args:
    d (dict): The dictionary to convert.
    
    Returns:
    str: The JSON string representation of the dictionary.
    """
    return json.dumps(d, sort_keys=True)

In [5]:
@lru_cache(maxsize=None)
def find_cheapest_operator(phone_number: str, preprocessed_lists: str) -> Tuple[Optional[str], float]:
    """
    Find the cheapest operator for a given phone number based on the preprocessed price lists.

    Args:
    phone_number (str): The phone number to find the cheapest operator for.
    preprocessed_lists (str): A JSON string representation of the preprocessed price lists.

    Returns:
    tuple: The name of the cheapest operator and the price per minute for the given phone number.
    """
    preprocessed_lists = json.loads(preprocessed_lists)
    best_matches = {}

    # Find the longest matching prefix for each operator
    for operator, prices in preprocessed_lists.items():
        longest_prefix = ""
        best_price = float('inf')
        for prefix, price in prices.items():
            if phone_number.startswith(prefix):
                if len(prefix) > len(longest_prefix) or (len(prefix) == len(longest_prefix) and price < best_price):
                    longest_prefix = prefix
                    best_price = price
        if longest_prefix:
            best_matches[operator] = (longest_prefix, best_price)

    # Compare the best matches to find the cheapest option
    cheapest_operator = None
    cheapest_price = float('inf')
    longest_prefix_length = 0

    for operator, (prefix, price) in best_matches.items():
        if price < cheapest_price or (price == cheapest_price and len(prefix) > longest_prefix_length):
            cheapest_operator = operator
            cheapest_price = price
            longest_prefix_length = len(prefix)

    return cheapest_operator, cheapest_price


In [6]:
# Prepare data once for reuse

cleaned_lists = clean_price_lists(price_lists)
preprocessed_lists = preprocess_price_lists(cleaned_lists)
preprocessed_lists_json_str = dict_to_json_str(preprocessed_lists)

Removed invalid entry from Operator A: prefix=-1, price=-0.5
Removed invalid entry from Operator A: prefix=nan, price=nan
Removed invalid entry from Operator B: prefix=-44, price=-0.3
Removed invalid entry from Operator B: prefix=nan, price=nan


In [7]:
def run_tests() -> None:
    """
    Run unit tests to verify the correctness of the find_cheapest_operator function.
    """
    tests = [
        {'phone_number': '4673212345', 'expected_operators': ['Operator A', 'Operator B'], 'expected_price': 1.1},
        {'phone_number': '4681234567', 'expected_operators': ['Operator A'], 'expected_price': 0.15},
        {'phone_number': '4631123456', 'expected_operators': ['Operator B'], 'expected_price': 0.14},
        {'phone_number': '4412345678', 'expected_operators': ['Operator B'], 'expected_price': 0.5},
        {'phone_number': '4812345678', 'expected_operators': ['Operator B'], 'expected_price': 1.2},
        {'phone_number': '2681234567', 'expected_operators': ['Operator A'], 'expected_price': 5.1},
        {'phone_number': '1234567890', 'expected_operators': ['Operator A'], 'expected_price': 0.9},
        {'phone_number': '467', 'expected_operators': ['Operator A'], 'expected_price': 0.21},
        {'phone_number': '46', 'expected_operators': ['Operator A'], 'expected_price': 0.21},
    ]

    for test in tests:
        operator, price = find_cheapest_operator(test['phone_number'], preprocessed_lists_json_str)
        assert operator in test['expected_operators'], f"Test failed for {test['phone_number']}: expected {test['expected_operators']}, got {operator}"
        assert price == test['expected_price'], f"Test failed for {test['phone_number']}: expected {test['expected_price']}, got {price}"
    print("All tests passed!")

In [8]:
if __name__ == "__main__":
    run_tests()
    phone_number = input("Enter the phone number: ")
    operator, price = find_cheapest_operator(phone_number, preprocessed_lists_json_str)
    if operator:
        print(f"The cheapest operator is {operator} with a price of ${price}/min.")
    else:
        print("No operator available for this phone number.")

All tests passed!
The cheapest operator is Operator A with a price of $0.21/min.


In [10]:
# Code below is responsible for recompiling the compiled Py code.

# !jupyter nbconvert --to script telephone_routing.ipynb

[NbConvertApp] Converting notebook telephone_routing.ipynb to script
[NbConvertApp] Writing 5909 bytes to telephone_routing.py
